---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    spath = []
    avgC = []
    dd = []
    method = []
    # Your Code Here
    for G in P1_Graphs:
        degree = G.degree()
        degree_values = sorted(set(degree.values()))
        shortest_path = nx.average_shortest_path_length(G)
        avg_clust = nx.average_clustering(G)
        spath.append(shortest_path)
        avgC.append(avg_clust)
        degd = [list(degree.values()).count(i) / float(nx.number_of_nodes(G)) for i in degree_values]
        dd.append(len(degd))
        if len(degd) > 10 :
            method.append('PA')
        elif avg_clust <0.1 :
            method.append('SW_H')
        else :
            method.append('SW_L')
    return method #spath, avgC, dd # Your Answer Here
         
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [10]:
def salary_predictions():
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_curve, auc, roc_auc_score
    from sklearn.preprocessing import MinMaxScaler
    
    # Turn G into a df 
    df = pd.DataFrame(index=G.nodes())
    scaler = MinMaxScaler()
    
    #Add features to your df 
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['degree'] = pd.Series(G.degree())
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pr'] = pd.Series(nx.pagerank(G))
    
    df_test = df[df['ManagementSalary'].isnull() == True]
    df_train = df[df['ManagementSalary'].isnull() == False]
    
    features = ['Department', 'degree', 'clustering', 'degree_centrality', 'closeness', 'betweeness', 'pr']
    
    df_Xtrain = df_train[features]
    ytrain = df_train['ManagementSalary']
    
    df_Xtest = df_test[features]
    
    #Test algorithms to see their AUC score 
    #X_train, X_test, y_train, y_test = train_test_split(df_Xtrain, df_Ytrain, random_state=0)
    
    #clf = LogisticRegression(C=100).fit(df_Xtrain, df_ytrain)
    #X_train_sc = scaler.fit_transform(X_train)
    #X_test_sc = scaler.transform(X_test)
    
    #clf = RandomForestClassifier(n_estimators=100, max_depth=10).fit(X_train_sc, y_train)
    #y_score = clf.predict(X_test_sc)
    #fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score)
    #roc_auc_lr = auc(fpr_lr, tpr_lr) # 0.6666
    #score = roc_auc_score(y_score, y_test) # 0.8777 - i think this is what they are using to grade
    
    Xtrain_sc = scaler.fit_transform(df_Xtrain)
    X_test_sc = scaler.transform(df_Xtest)
    
    clf = RandomForestClassifier(n_estimators=100, max_depth=10).fit(Xtrain_sc, ytrain)
    
    pred_proba = clf.predict_proba(X_test_sc)[:, 1]
    return pd.Series(pred_proba, df_Xtest.index)# Your Answer Here
salary_predictions()

1       0.016535
2       0.963333
5       1.000000
8       0.379309
14      0.045605
18      0.085398
27      0.043811
30      0.734829
31      0.371084
34      0.130811
37      0.125471
40      0.075179
45      0.003482
54      0.399135
55      0.548771
60      0.307064
62      1.000000
65      0.960000
77      0.032860
79      0.018850
97      0.000000
101     0.000526
103     0.445424
108     0.019300
113     0.124564
122     0.000000
141     0.301479
142     1.000000
144     0.010000
145     0.519029
          ...   
913     0.006806
914     0.002500
915     0.010000
918     0.101265
923     0.018690
926     0.061190
931     0.000000
934     0.000000
939     0.000000
944     0.000278
945     0.005000
947     0.041828
950     0.019650
951     0.010000
953     0.005000
959     0.000000
962     0.000000
963     0.125354
968     0.001471
969     0.007282
974     0.000526
984     0.010000
987     0.000883
989     0.001471
991     0.021471
992     0.000000
994     0.000278
996     0.0002

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [11]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})

future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [35]:
def new_connections_predictions():
    
    # Your Code Here
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_curve, auc, roc_auc_score
    from sklearn.preprocessing import MinMaxScaler
    
    # Turn G into a df 
    
    scaler = MinMaxScaler()
    future_connections['Origin'] = [orgs for (orgs, des) in future_connections.index]
    future_connections['Destination'] = [des for (orgs, des) in future_connections.index]
    future_connections['Edge'] = future_connections.index
    future_connections['Origin degree'] = [G.degree(node) for node in future_connections['Origin']]
    future_connections['Destination degree'] = [G.degree(node) for node in future_connections['Destination']]
    future_connections['Preferential Attachment'] = [list(nx.preferential_attachment(G, [edge]))[0][2] for edge in future_connections['Edge']]
    future_connections['Common Neighbors'] = [list(nx.common_neighbors(G, edge[0], edge[1])) for edge in future_connections['Edge']]
    future_connections['Num of Common Neighbors'] = [len(neighbors) for neighbors in future_connections['Common Neighbors']]
    future_connections['Jaccard Coefficient'] = [list(nx.jaccard_coefficient(G, [edge]))[0][2] for edge in future_connections['Edge']]
    future_connections['Resource Allocation Index'] = [list(nx.resource_allocation_index(G, [edge]))[0][2] for edge in future_connections['Edge']]
    future_connections['Adamic Adar Index'] = [list(nx.adamic_adar_index(G, [edge]))[0][2] for edge in future_connections['Edge']]
    future_connections['CN Soundarajan Hopcroft'] = [list(nx.cn_soundarajan_hopcroft(G, ebunch=[edge], community='Department'))[0][2] for edge in future_connections['Edge']]
    future_connections['RA Soundarajan Hopcroft'] =  [list(nx.ra_index_soundarajan_hopcroft(G, ebunch=[edge], community='Department'))[0][2] for edge in future_connections['Edge']]
    #future_connections['Origin Degree'] = [node for G.degree(node) in future_connections['Origin']]
    
    df_test = future_connections[future_connections['Future Connection'].isnull() == True]
    df_train = future_connections[future_connections['Future Connection'].isnull() == False]
    
    features = ['Origin degree', 'Destination degree', 'Preferential Attachment', 'Num of Common Neighbors', 'Jaccard Coefficient', 'Resource Allocation Index', 'Adamic Adar Index', 'CN Soundarajan Hopcroft', 'RA Soundarajan Hopcroft']
    
    df_Xtrain = df_train[features]
    ytrain = df_train['Future Connection']
    
    df_Xtest = df_test[features]
    
    Xtrain_sc = scaler.fit_transform(df_Xtrain)
    X_test_sc = scaler.transform(df_Xtest)
    
    clf = RandomForestClassifier(n_estimators=100, max_depth=10).fit(Xtrain_sc, np.ravel(ytrain))
    
    pred_proba = clf.predict_proba(X_test_sc)[:, 1]
    
    return pd.Series(pred_proba, df_Xtest.index) #future_connections# Your Answer Here
new_connections_predictions()

(107, 348)    0.026821
(542, 751)    0.012183
(20, 426)     0.571214
(50, 989)     0.012086
(942, 986)    0.012370
(324, 857)    0.012147
(13, 710)     0.116520
(19, 271)     0.148610
(319, 878)    0.012092
(659, 707)    0.012655
(49, 843)     0.012233
(208, 893)    0.012236
(377, 469)    0.015824
(405, 999)    0.015939
(129, 740)    0.012940
(292, 618)    0.018744
(239, 689)    0.012092
(359, 373)    0.014010
(53, 523)     0.417003
(276, 984)    0.012127
(202, 997)    0.012583
(604, 619)    0.032085
(270, 911)    0.012076
(261, 481)    0.060927
(200, 450)    0.981445
(213, 634)    0.012747
(644, 735)    0.033361
(346, 553)    0.012651
(521, 738)    0.011812
(422, 953)    0.015477
                ...   
(672, 848)    0.012076
(28, 127)     0.959686
(202, 661)    0.012350
(54, 195)     0.999970
(295, 864)    0.013938
(814, 936)    0.013109
(839, 874)    0.012370
(139, 843)    0.013176
(461, 544)    0.012907
(68, 487)     0.013644
(622, 932)    0.012028
(504, 936)    0.017683
(479, 528) 